**Общая информация**

**Срок сдачи:** 13 марта 2017, 06:00 <br\>
**Штраф за опоздание:** -2 балла после 06:00 13 марта, -4 балла после 06:00 20 марта, -6 баллов после 06:00 27 марта

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush

Необходимо в slack создать таск в приватный чат:
/todo Фамилия Имя ссылка на гитхаб @alkhamush
Пример:
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе.

###### Задание 2 (2 балла)
Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn 4 балла для iris и mnist
Для этого используем numpy

###### Задание 3 (2 балла)
Для iris найдите такой параметр n_neighbors, при котором выдаётся наилучший score. Нарисуйте график зависимости score от n_neighbors

###### Задание 3 (2 балла)
Выполнить требования pep8

###### Задание 5 (2 балла)
Описать для чего нужны следующие библиотеки/классы/функции (список будет ниже)


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata
from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, \
    SupervisedIntegerMixin, RadiusNeighborsMixin
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics
from scipy import stats
#%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle

class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin, 
                             SupervisedIntegerMixin, ClassifierMixin):
    """KNN class. Metric = euclid"""
    
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
    
    def _getKNeighborsClasses(self, X, n_neighbors=None):
        """returns k nearest neighbors classes"""
        if n_neighbors is None:
            n_neighbors = self.n_neighbors
            
        pairw_dist = sklearn.metrics.pairwise.pairwise_distances(X, self.X_)
        neigborsIndexes = np.argsort(pairw_dist, axis=1)[:, : n_neighbors]
        return self.y_[neigborsIndexes]
    
    def fit(self, X, y):
        """X - train data, X = np.ndarray with dim 1 or 2
        y - target values, y = np.ndarray with dim 1
        """
        if X.ndim > 2 or y.ndim != 1:
            raise Exeption("invalid data format of X, y")
        
        if X.ndim == 1:
            self.X_ = X.reshape(X.shape[0], 1)
        else:
            self.X_ = X
    
        self.y_ = y
        self.classes_ = np.unique(y)        
    
    def predict(self, X):
        """X - np.ndarray, shape (n_query, n_features)
        returns array of shape [n_samples]
        """
        if X.ndim != 2 or X.shape[1] != self.X_.shape[1]:
            raise Exeption("invalid data format of X")
            
        classesNeighbors = self._getKNeighborsClasses(X)
        return np.array(stats.mode(classesNeighbors, axis=1))[0].ravel()
    
    def predict_proba(self, X):
        classesNeighbors = self._getKNeighborsClasses(X)
        
        probs = np.array([])
        
        for current_cl in self.classes_:
            np.hstack((probs, np.sum(current_cl == classesNeighbors, axis=1)))

        return probs.T / self.n_neighbors

    
    def score(self, X, y):
        return np.average(self.predict(X) == y)

**IRIS**

In [18]:
iris = datasets.load_iris()

In [19]:
#X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target, random_state = 45)

In [25]:
clf = KNeighborsClassifier(n_neighbors=4)
my_clf = MyKNeighborsClassifier(n_neighbors=4)

In [26]:
%time clf.fit(X_train, y_train)

Wall time: 1 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=4, p=2,
           weights='uniform')

In [27]:
%time my_clf.fit(X_train, y_train)

Wall time: 0 ns


In [28]:
%time clf.predict(X_test)

Wall time: 1.88 ms


array([1, 1, 0, 1, 0, 2, 1, 2, 1, 1, 0, 0, 0, 1, 2])

In [29]:
%time my_clf.predict(X_test)

Wall time: 2 ms


array([1, 1, 0, 1, 0, 2, 1, 2, 1, 1, 0, 0, 0, 1, 2])

In [30]:
%time clf.predict_proba(X_test)

Wall time: 2.01 ms


array([[0.  , 0.5 , 0.5 ],
       [0.  , 1.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  ],
       [0.  , 1.  , 0.  ],
       [0.  , 0.  , 1.  ],
       [0.  , 1.  , 0.  ],
       [0.  , 1.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [0.  , 0.75, 0.25],
       [0.  , 0.  , 1.  ]])

In [31]:
%time my_clf.predict_proba(X_test)

Wall time: 2 ms


array([], dtype=float64)

In [32]:
clf.score(X_test, y_test)

0.8666666666666667

In [33]:
my_clf.score(X_test, y_test)

0.8666666666666667

In [ ]:
# Задание 3

err_train = np.array([])
err_test = np.array([])
neighbors_range = np.arange(2, 40)

for k in neighbors_range:
    my_knn = MyKNeighborsClassifier(k)
    my_knn.fit(X_train, y_train)

    np.hstack((err_train, my_knn.score(X_train, y_train)))
    np.hstack((err_test, my_knn.score(X_test, y_test)))

plt.plot(neighbors_range, err_train, color='blue', label='train')
plt.plot(neighbors_range, err_test, color='red', label='test')
plt.legend()
plt.show()
print("\n\nМаксимум score достигается "
      "при поиске ", np.argmax(err_test), " соседей")

**MNIST**

In [34]:
mnist = fetch_mldata('MNIST original')

In [35]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.1, stratify=mnist.target)

In [36]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [37]:
%time clf.fit(X_train, y_train)

Wall time: 1min 19s


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [38]:
%time my_clf.fit(X_train, y_train)

Wall time: 3.88 ms


In [39]:
%time clf.predict(X_test)

Wall time: 20min 16s


array([0., 0., 2., ..., 6., 8., 1.])

In [40]:
%time my_clf.predict(X_test)

Wall time: 2min 19s


array([0., 0., 2., ..., 6., 8., 1.])

In [41]:
%time clf.predict_proba(X_test)

Wall time: 20min 39s


array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [42]:
%time my_clf.predict_proba(X_test)

Wall time: 2min 12s


array([], dtype=float64)

In [43]:
clf.score(X_test, y_test)

0.9687142857142857

In [44]:
my_clf.score(X_test, y_test)

0.9687142857142857

Задание 5

In [ ]:
# seaborn
"""
библиотека, служащая для визуализации данных. Основана на matplotlib.
Являет собой высокоуровневый интерфейс для построения графиков статистических данных
"""
# matplotlib
"""
библиотека для визуализации данных при помощи 2D (и 3D) графики
"""
# train_test_split
"""
разбивает массив данных на тренировочную и тестовую части
"""
# Pipelin%load_ext e (from sklearn.pipeline import Pipeline)
"""
соединяет воедино несколько методов или алгоритмов, что необходимо для
автоматизации работы с массивами данных
"""
# StandardScaler (from sklearn.preprocessing import StandardScaler)
"""
стандартизирует признаки, вычитая среднее и перенормируя так, чтобы дисперсия была единичной
"""
# ClassifierMixin
"""
класс mixin с методом score, выполняющим количественную оценку качества классификации
"""
# NeighborsBase
"""
базовый класс для всех методов k ближайших соседей
"""
# KNeighborsMixin
"""
класс-mixin для поиска k ближайших соседей. Метод kneighbors находит k соседей,
а kneighbors_graph строит их граф
"""
# SupervisedIntegerMixin
"""
класс-mixin с методом fit(self, X, y), служащий для обучения модели с X - train data и y - target values
"""